# Imports and configuration

In [ ]:
# Imports
import io
import math
import os

import folium
import geojson
import numpy as np
import pandas as pd
import seaborn as sns
from PIL import Image

In [ ]:
# Change directory to root
directory = os.getcwd().split('/')[-1]

if directory == 'notebooks':
    %cd ..

In [ ]:
# Matplotlib import and setup
import matplotlib

# matplotlib.use('PDF')

import matplotlib.pyplot as plt
print(f'matplotlib backend: {matplotlib.get_backend()}')

In [ ]:
# configuration variables
should_save = False

In [ ]:
# Import functions
from coordinate_converter import utm_to_longitude_latitude
import map_utils


In [ ]:
incidents = pd.read_csv('proprietary_data/processed_data.csv', index_col=0)
incidents

# Build heatmap

In [ ]:
counts = incidents.groupby(['xcoor', 'ycoor'], as_index=False).size()
counts['counts'] = counts['size']
counts.drop(['size'], axis=1, inplace=True)

empty_cells = pd.read_csv('data/empty_cells.csv', encoding='utf-8', index_col=0)
empty_cells = empty_cells[['X', 'Y']].rename(columns={'X': 'xcoor', 'Y': 'ycoor'})
counts = pd.concat([counts, empty_cells.assign(counts=0)]) 

counts.sort_values('counts')

## Save as geojson file

In [ ]:
features = counts.apply(lambda row: map_utils.centroid_to_geojson_square(*row), axis=1).tolist()
geojson_file_name = 'data/grid.geojson'

with open(geojson_file_name, 'w', encoding='utf8') as file:
        geojson.dump(geojson.FeatureCollection(features), file)

## Render map and save to image

In [ ]:
if should_save:
    
    folium_map = folium.Map(
        width=400,
        location=[60.3, 8.6],
        tiles='cartodbpositron',
        zoom_start=8,
        zoom_control=False,
    )
    
    folium.GeoJson(geojson_file_name, name='geojson', style_function=map_utils.style_function).add_to(folium_map)

    # Center map on data
    south_west, north_east = list(zip(counts.xcoor.agg(['min', 'max']), counts.ycoor.agg(['min', 'max'])))
    south_west, north_east = utm_to_longitude_latitude(south_west)[::-1], utm_to_longitude_latitude(north_east)[::-1]

    folium_map.fit_bounds([south_west, north_east])

    image_data = folium_map._to_png(delay=7)
    heatmap = Image.open(io.BytesIO(image_data))
    heatmap.save('../output/visualization/heatmap.png')

    map_utils.plot_colors_as_legend(title='Aggregated incidents', file_path='../output/visualization/heatmap_legend.png')

## Number of grid cells per category

In [ ]:
count_logs = counts[counts.counts > 0]['counts'].apply(np.log10)
log_groups = pd.DataFrame(count_logs.apply(math.floor).value_counts())
log_groups = pd.concat([pd.DataFrame([counts[counts.counts == 0]['counts'].shape[0]], columns=['counts']), log_groups], ignore_index=True)

color_map = map_utils.get_colors()
log_groups.index=color_map.keys()
edgecolors = ['#00000000'] *  len(color_map)
edgecolors[0] = 'grey'

ax = sns.barplot(data=log_groups, x=log_groups.counts, y=log_groups.index, palette=color_map.values(), edgecolor=edgecolors)
ax.set(xlabel='Number of grid cells', ylabel='Number of incidents')

bars = ax.patches
labels = log_groups.counts

for bar, label in zip(bars, labels):
    width = bar.get_width()
    height = bar.get_height()
    y = bar.get_y()
    ax.text(width + 150, y + height / 2, label, ha='center')

sns.despine()
plt.tight_layout()

if should_save:
    plt.savefig('../output/visualization/number_of_grid_cells.pdf', dpi=600)

# Base station clustering

In [ ]:
grids = pd.read_csv('data/grid_zones.csv', index_col=0)
grids

In [ ]:
empty_cells = pd.read_csv('data/empty_cells.csv', encoding='utf-8', index_col=0)
empty_cells = empty_cells[['X', 'Y']].rename(columns={'X': 'easting', 'Y': 'northing'})
empty_cells['easting'] = empty_cells['easting'].astype(int)
empty_cells['northing'] = empty_cells['northing'].astype(int)

grids = grids[['easting', 'northing', 'base_station']]

grids = pd.concat([grids, empty_cells.assign(base_station=19)])
grids

In [ ]:
base_stations = pd.read_csv('data/base_stations.csv', encoding='utf-8', index_col=0)
base_stations = base_stations[['easting', 'northing']]
base_stations

In [ ]:
geojson_grid_zones_file = 'data/grid_zones.geojson'

squares = grids.apply(lambda row: map_utils.centroid_to_geojson_square(*row, color_mapper=map_utils.zone_color), axis=1).tolist()
points = base_stations.apply(lambda row: map_utils.centroid_to_geojson(*row), axis=1).tolist()

features = squares # + points

with open(geojson_grid_zones_file, 'w', encoding='utf8') as file:
        geojson.dump(geojson.FeatureCollection(features), file)
    
folium_map = folium.Map(
    width=400,
    location=[60.3, 8.6],
    tiles='cartodbpositron',
    zoom_start=8,
    zoom_control=False,
)

folium.GeoJson(geojson_grid_zones_file, name='geojson', style_function=map_utils.style_function).add_to(folium_map)

for point in points:

    folium.CircleMarker(
        location=point['geometry']['coordinates'][::-1],
        radius=3,
        color='#000000',
        fill=True,
        fill_color='#000000',
    ).add_to(folium_map)

# Center map on data
south_west, north_east = list(zip(grids.easting.agg(['min', 'max']), grids.northing.agg(['min', 'max'])))
south_west, north_east = utm_to_longitude_latitude(south_west)[::-1], utm_to_longitude_latitude(north_east)[::-1]

folium_map.fit_bounds([south_west, north_east])

image_data = folium_map._to_png(delay=7)
zones_image = Image.open(io.BytesIO(image_data))

# Crop image
width, height = zones_image.size
zones_image = zones_image.crop((0, 0, 400, height))

zones_image.save('../output/visualization/grid_zones.png')

In [ ]:
import coordinate_converter

folium_map = folium.Map(
    width=400,
    location=[60.3, 8.6],
    tiles='cartodbpositron',
    zoom_start=10,
    zoom_control=False,
)

example_allocation = [2, 1, 2, 4, 4, 3, 1, 4, 0, 2, 2, 3, 2, 2, 3, 3, 3, 1, 3]

locations = base_stations.apply(lambda row: coordinate_converter.utm_to_longitude_latitude(row)[::-1], axis=1).tolist()

for i, location in enumerate(locations):
    if (example_allocation[i] != 0):
        folium.plugins.MarkerCluster(locations=([location] * example_allocation[i])).add_to(folium_map)

# Center map on data
south_west, north_east = list(zip(grids.easting.agg(['min', 'max']), grids.northing.agg(['min', 'max'])))
south_west, north_east = utm_to_longitude_latitude(south_west)[::-1], utm_to_longitude_latitude(north_east)[::-1]

folium_map.fit_bounds([south_west, north_east])

image_data = folium_map._to_png(delay=7)
zones_image = Image.open(io.BytesIO(image_data))

# Crop image
width, height = zones_image.size
zones_image = zones_image.crop((0, 0, 400, height))

zones_image.save('../output/visualization/allocation.png')